# Import packages

In [ ]:
import os
import cv2
import cmaps
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from matplotlib.backends.backend_pdf import PdfPages
version = mpl.__version__
rcParams['font.family'] = 'sans-serif'
directory   = '/srv/scratch/z3533156'

# Import time functions

In [ ]:
def datestring_to_serial_day(datestring,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import pandas as pd
    import datetime
    serial_day_timedelta = pd.to_datetime(datestring) - datetime.datetime(epochY,epochm,epochd,epochH,epochM)
    corrected_serial_day_number = serial_day_timedelta.days + serial_day_timedelta.seconds/86400
    return corrected_serial_day_number
def serial_day_to_datestring(day,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import datetime
    corrected_date = datetime.datetime(epochY,epochm,epochd,epochH,epochM) + datetime.timedelta(day)
    return corrected_date.strftime("%Y-%m-%d") 
def serial_day_to_datestring1(day,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import datetime
    corrected_date = datetime.datetime(epochY,epochm,epochd,epochH,epochM) + datetime.timedelta(day)
    return corrected_date.strftime("%Y-") 
def serial_day_to_datestring2(day,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import datetime
    corrected_date = datetime.datetime(epochY,epochm,epochd,epochH,epochM) + datetime.timedelta(day)
    return corrected_date.strftime("%m-%d")  

# Read data

In [ ]:
AVISO_SLA   = Dataset(directory+'/MHW/AVISO/AVISO_EAC_MHW.nc','r')
SLA_daily   = np.transpose(AVISO_SLA.variables['sla'],[2,1,0])
SSH_daily   = np.transpose(AVISO_SLA.variables['adt'],[2,1,0])
lon0        = AVISO_SLA.variables['lon'][:]
lat0        = AVISO_SLA.variables['lat'][:]
lon         = np.tile(lon0,[np.size(lat0,0),1]).transpose()
lat         = np.tile(lat0,[np.size(lon0,0),1])
dataset     = sio.loadmat(directory+'/MHW/Figure_data/Figure1_OSTIA_MHW.mat')
time_daily  = dataset['time_daily'][:,0]/86400
time        = datestring_to_serial_day("1981-01-01",epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0)
daily_t     = np.arange(date(1993,1,1).toordinal(),date(2022,12,31).toordinal()+1,1)
daily_t1    = np.arange(date(2021,12,1).toordinal(),date(2022,2,28).toordinal()+1,1)
daily_dates = [date.fromordinal(tt.astype(int)) for tt in daily_t1]
SLA         = SLA_daily[:,:,(daily_t>=date(2021,12,1).toordinal()) & (daily_t<=date(2022,2,28).toordinal())]
SSH         = SSH_daily[:,:,(daily_t>=date(2021,12,1).toordinal()) & (daily_t<=date(2022,2,28).toordinal())]
dataset1       = sio.loadmat(directory+'/MHW/Figure_data/Figure_etopo_AVISO.mat')
depth          = dataset1['depth'][:,:]
depth[lon>155] = np.nan

# Plot the spatial distribution of weekly sea level anomalies

In [ ]:
fig = plt.figure(figsize=(80, 60))
gs  = gridspec.GridSpec(3,4) 
m   = Basemap(projection='merc',llcrnrlat=-38.0-0.001,urcrnrlat=-28+0.001,llcrnrlon=150.0-0.001,urcrnrlon=160-0.001,resolution='i') 
x2, y2 = m(lon, lat) 
padspacescale = 50 
labelpadscale = 15 
linefont  = 12 
labelfont = 70 
scale1    = 1.35 
scale2    = 1.2 
levels1   = np.linspace(-0.5,0.5,50) 
levels2   = np.linspace(-100,100,50) 
tick_marks1 = np.linspace(-0.5,0.5,11) 
tick_marks2 = np.linspace(-100,100,9) 
sla_levels1 = np.linspace(0,0.5,3) 
sla_levels2 = np.linspace(-0.5,0,3) 
sla_levels3 = np.linspace(-0.001,0.001,2) 
ssh_levels1 = np.linspace(0.949,0.951,2) 
cmaps1 = cmaps.cmocean_balance 
labels = ['a','b','c','d','e','f','g','h','i','j','k','l'] 
for i in range(12): 
    k = i
    if i<4: 
        j = np.mod(i,4) 
    elif i<8: 
        j = 4 + np.mod(i,4) 
    else: 
        j = 8 + np.mod(i,4)
    ax = fig.add_subplot(gs[j]) 
    l, b, w, h = ax.get_position().bounds 
    m = Basemap(projection='merc',llcrnrlat=-38.0-0.001,urcrnrlat=-28+0.001,llcrnrlon=150.0-0.001,urcrnrlon=160-0.001,resolution='i') 
    m.drawcoastlines(color='0.1', linewidth=0.5*linefont)
    m.drawmapboundary(color='0.1', linewidth=0.5*linefont) 
    m.fillcontinents(color='0.95', lake_color='white') 
    m.drawmeridians(np.arange(150,165,2),labels=[0,0,0,0],linewidth=0.4*linefont,dashes=[2,2],color='gray',fontsize=1.1*labelfont)
    CB1 = m.contourf(x2, y2, SLA[:,:,k], 
    cmap=cmaps1,levels=levels1,origin='lower',extend='both')
    CS1 = m.contour(x2, y2, SSH[:,:,k], ssh_levels1,linewidths=1.2*linefont,linestyles='solid',colors='black') 
    CS2 = m.contour(x2, y2, SLA[:,:,k], sla_levels3,linewidths=1.2*linefont,linestyles='solid',colors='gray') 
    CS3 = m.contour(x2, y2, depth, np.arange(-201,-199,10), linewidths=1.2*linefont,linestyles='dashed',colors='xkcd:purple')
    lon2 = np.array([lon[22,20],lon[60,20],lon[60,33],lon[30,33],lon[22,20]]) 
    lat2 = np.array([lat[22,20],lat[60,20],lat[60,33],lat[30,33],lon[22,20]]) 
    x22,y22 = m(lon2,lat2) 
    lon3 = np.array([lon[22,20],lon[25,20],lon[33,33],lon[30,33],lon[22,20]]) 
    lat3 = np.array([lat[22,20],lat[25,20],lat[33,33],lat[30,33],lat[22,20]]) 
    x33,y33 = m(lon3,lat3) 
    m.plot(x33, y33, linewidth=labelpadscale, linestyle='dashed', color='green') 
    if np.mod(i,4)==0: 
        plt.ylabel('Latitude',fontsize=labelfont,labelpad=3.5*padspacescale,family='sans-serif') 
        m.drawparallels(np.arange(-40,-10, 2),labels=[1,0,0,0],linewidth=0.4*linefont,dashes=[2,2],color='gray',fontsize=1.1*labelfont) 
    elif np.mod(i,4)==1: 
        l=l-0.006 
        m.drawparallels(np.arange(-40,-10, 2),labels=[0,0,0,0],linewidth=0.4*linefont,dashes=[2,2],color='gray',fontsize=1.1*labelfont) 
    elif np.mod(i,4)==2: 
        l=l-0.012 
        m.drawparallels(np.arange(-40,-10, 2),labels=[0,0,0,0],linewidth=0.4*linefont,dashes=[2,2],color='gray',fontsize=1.1*labelfont) 
    else: 
        l=l-0.018 
        m.drawparallels(np.arange(-40,-10, 2),labels=[0,0,0,0],linewidth=0.4*linefont,dashes=[2,2],color='gray',fontsize=1.1*labelfont) 
    ax.spines['bottom'].set_linewidth(labelpadscale) 
    ax.spines['left'].set_linewidth(labelpadscale) 
    ax.spines['top'].set_linewidth(labelpadscale) 
    ax.spines['right'].set_linewidth(labelpadscale) 
    cx1,cy1 = m(150.2, -28.8) 
    cx2,cy2 = m(150.2, -29.6) 
    plt.text(cx1, cy1,serial_day_to_datestring1(time + time_daily[10561+k]), color='dodgerblue', fontsize=1.2*labelfont, weight='bold',family='sans-serif') 
    plt.text(cx2, cy2,serial_day_to_datestring2(time + time_daily[10561+k]), color='dodgerblue', fontsize=1.2*labelfont, weight='bold',family='sans-serif') 
    if i<4: 
        b = b 
    elif i<8: 
        b = b-0.06 
    else:
        b = b-0.12 
    plt.xlabel('Longitude',fontsize=labelfont,labelpad=1.5*padspacescale,family='sans-serif') 
    m.drawmeridians(np.arange(150,165, 4),labels=[0,0,0,1],linewidth=0.4*linefont,dashes=[2,2],color='.7',fontsize=1.1*labelfont)
    ax.set_position([l, b, scale1*w, scale1*h])
    plt.tick_params(axis='both',which='major',bottom='on',left='on',length=40.0,width=15,colors='black',direction='out') 
    plt.title(labels[i], fontsize=1.2*labelfont,loc='left',pad=0.6*padspacescale, weight='bold',family='sans-serif') 
    cbaxes1 = fig.add_axes([0.15, -0.08, 0.75, 0.02]) 
    cb1 = plt.colorbar(CB1,orientation='horizontal',cax = cbaxes1) 
    cb1.set_ticks(tick_marks1) 
    cb1.set_label(r'Sea level anomalies (m)', fontsize=labelfont,labelpad=0) 
    cb1.ax.tick_params(labelsize=labelfont) 
#############################################################################################
fig.savefig(directory+'/MHW/Figure_plots/Figure5_weekly_SLA.png',dpi=300,bbox_inches = 'tight')